# Calculate potential Evapo-transpiration (PET) girds
The purpose of this notebook is to create PET grids as part of the Cambodia/eWater project. The notebook uses daily min, mean and max air temperature grids and the Hargreaves equation.

author: Erin Telfer (erin.telfer@ga.gov.au) published: 10/07/2018
Code comes with no warranty. If you have suggestions/improvement/bug finds please email.


## Import Libraries

In [1]:
import numpy as np
import math as mt
from datetime import *
import calendar
import netCDF4 as nc
import xarray as xr
import glob

## Create Functions

In [2]:
#Create function to calculate date from netcdf (as units of netcdf are "seconds since 20xx-xx-xx 00:00:00" )
def to_date(epoch, time_unit):
    time_unit =  datetime.strptime(time_unit, '%Y-%m-%d %H:%M:%S') 
    return time_unit + timedelta(seconds=epoch)

#Create function to calulate julian day of year (days since start of year)
def julian_days(epoch, time_unit):
    start_of_year = datetime.strptime('01/01/'+ str(time_unit),'%m/%d/%Y')
    start_of_month = datetime.strptime(min_xr.time.units[14:], '%Y-%m-%d %H:%M:%S') 
    epoch = start_of_month + timedelta(seconds=epoch)
    j_days = int((((epoch - start_of_year)+timedelta(days=1))/timedelta(days=1)))
    return (j_days)

#Create function to test whether the year of interest is a leap year
def leap_year(year):
    if calendar.isleap(int(year)) == True:
#         print(str(year) +' is a leap year')
        ndy = 366
    else:
#         print(str(year) +' is not a leap year')
        ndy = 365
    return(ndy)
    

## loop through all temperature grids and save as netCDF format

In [3]:
input_folder = '/g/data/u46/users/ext547/ewater/input_data/laos/temp/' #location of files
ncfiles = glob.glob(input_folder + '/celcius/*.nc') #a glob file containing the names of all nc

print(ncfiles)
for ncpath in ncfiles: #loop through all nc
    #Set dataset variables
    nc_name = ncpath[-28:]
    print ('Processing '+str(nc_name)+' now')
    min_nc = 'min_'+str(nc_name)
    max_nc = 'max_'+str(nc_name)
    mean_nc = 'mean_'+str(nc_name)
    
    #Input temperature datasets
    min_xr = xr.open_dataset(input_folder+'/min/'+min_nc, decode_times=False)
    mean_xr = xr.open_dataset(input_folder+'/mean/'+mean_nc, decode_times=False)
    max_xr = xr.open_dataset(input_folder+'/max/'+max_nc, decode_times=False)
    
    #Calculate date from netcdf and save as xarray dataset
    time = xr.DataArray([to_date(epoch, min_xr.time.units[14:]) for epoch in min_xr.time.values.tolist()], coords={'time': min_xr.time}, dims=('time',))
   
    #Calculate julian days and save as an array
    j_days = xr.DataArray([julian_days(epoch, 
                                       min_xr.time.units[14:18]) for epoch in time.time.values.tolist()], 
                          coords={'time': min_xr.time}, dims=('time',))
    
    #Use leap year function and save the number of days within the year as a variable
    ndy = leap_year( min_xr.time.units[14:18])
    
    #Create array of temperature difference
    #Unit: T_diff in degrees celcius
    T_diff = max_xr.Tair - min_xr.Tair
    
    #Create array of latitude in radians
    #Unit: lat_rad in radians
    lat_rad = np.radians(min_xr.lat)
    
    # Caclulate the inverse relative distance between the Earth and the Sun
    # Unit: dr in radians
    dr = 1 + 0.033 * np.cos(2 * np.pi * j_days / ndy)
    
    # Calculate solar declination
    # Unit: sd in radians
    sd = 0.409 * np.sin(2 * mt.pi * j_days / ndy - 1.39)

    # Calculate a sunset angle input variable
    # Unit: no unit
    sa_X = 1 - (np.tan(lat_rad))**2 * ((np.tan(sd))**2)
    
    # Calculate sunset angle
    # Units: sa in radians
    sa = mt.pi / 2 - np.arctan(-np.tan(lat_rad) * np.tan(sd) / np.sqrt(sa_X))

    # Calcualte extraterrestrial radiation
    # Units: ra in MJ / (m^ day)
    ra = 37.586 * dr * (sa * np.sin(lat_rad) * np.sin(sd) + np.cos(lat_rad) * np.cos(sd) * np.sin(sa))
    
    # Calculate Potential Evapotranspiration
    # Units: PET in mm/day
    pet = 0.0009384 * (mean_xr.Tair + 17.8) * np.sqrt(T_diff) * ra
    pet = xr.Dataset({'PET':pet})
    pet.PET.attrs['units'] = 'mm/day' #add unit to xarray
    
    #Save PET output as netcdf
    pet.to_netcdf(path = '/g/data/u46/users/ext547/ewater/input_data/laos/PET/monthly/PET'+nc_name[12:])
    print ('PET'+str(nc_name[12:])+' is complete')

['/g/data/u46/users/ext547/ewater/input_data/laos/temp//celcius/celsius_Tair_WFDEI_198902.nc', '/g/data/u46/users/ext547/ewater/input_data/laos/temp//celcius/celsius_Tair_WFDEI_199102.nc', '/g/data/u46/users/ext547/ewater/input_data/laos/temp//celcius/celsius_Tair_WFDEI_198310.nc', '/g/data/u46/users/ext547/ewater/input_data/laos/temp//celcius/celsius_Tair_WFDEI_200904.nc', '/g/data/u46/users/ext547/ewater/input_data/laos/temp//celcius/celsius_Tair_WFDEI_199609.nc', '/g/data/u46/users/ext547/ewater/input_data/laos/temp//celcius/celsius_Tair_WFDEI_201608.nc', '/g/data/u46/users/ext547/ewater/input_data/laos/temp//celcius/celsius_Tair_WFDEI_199207.nc', '/g/data/u46/users/ext547/ewater/input_data/laos/temp//celcius/celsius_Tair_WFDEI_198012.nc', '/g/data/u46/users/ext547/ewater/input_data/laos/temp//celcius/celsius_Tair_WFDEI_201205.nc', '/g/data/u46/users/ext547/ewater/input_data/laos/temp//celcius/celsius_Tair_WFDEI_198408.nc', '/g/data/u46/users/ext547/ewater/input_data/laos/temp//celc

PET_WFDEI_199102.nc is complete
Processing celsius_Tair_WFDEI_198310.nc now
PET_WFDEI_198310.nc is complete
Processing celsius_Tair_WFDEI_200904.nc now
PET_WFDEI_200904.nc is complete
Processing celsius_Tair_WFDEI_199609.nc now
PET_WFDEI_199609.nc is complete
Processing celsius_Tair_WFDEI_201608.nc now
PET_WFDEI_201608.nc is complete
Processing celsius_Tair_WFDEI_199207.nc now
PET_WFDEI_199207.nc is complete
Processing celsius_Tair_WFDEI_198012.nc now
PET_WFDEI_198012.nc is complete
Processing celsius_Tair_WFDEI_201205.nc now
PET_WFDEI_201205.nc is complete
Processing celsius_Tair_WFDEI_198408.nc now
PET_WFDEI_198408.nc is complete
Processing celsius_Tair_WFDEI_199803.nc now
PET_WFDEI_199803.nc is complete
Processing celsius_Tair_WFDEI_200212.nc now
PET_WFDEI_200212.nc is complete
Processing celsius_Tair_WFDEI_201009.nc now
PET_WFDEI_201009.nc is complete
Processing celsius_Tair_WFDEI_200304.nc now
PET_WFDEI_200304.nc is complete
Processing celsius_Tair_WFDEI_198609.nc now
PET_WFDEI_19

PET_WFDEI_200102.nc is complete
Processing celsius_Tair_WFDEI_200002.nc now
PET_WFDEI_200002.nc is complete
Processing celsius_Tair_WFDEI_201607.nc now
PET_WFDEI_201607.nc is complete
Processing celsius_Tair_WFDEI_200109.nc now
PET_WFDEI_200109.nc is complete
Processing celsius_Tair_WFDEI_199701.nc now


KeyboardInterrupt: 

In [44]:
j_days.isel(time=29)

<xarray.DataArray ()>
array(273)
Coordinates:
    time     float64 2.543e+06

In [42]:
pet.time

<xarray.DataArray 'time' (time: 30)>
array([  37800.,  124200.,  210600.,  297000.,  383400.,  469800.,  556200.,
        642600.,  729000.,  815400.,  901800.,  988200., 1074600., 1161000.,
       1247400., 1333800., 1420200., 1506600., 1593000., 1679400., 1765800.,
       1852200., 1938600., 2025000., 2111400., 2197800., 2284200., 2370600.,
       2457000., 2543400.])
Coordinates:
  * time     (time) float64 3.78e+04 1.242e+05 2.106e+05 ... 2.457e+06 2.543e+06
Attributes:
    standard_name:  time
    long_name:      time since start of month
    bounds:         time_bnds
    units:          seconds since 2001-09-01 00:00:00
    calendar:       standard
    axis:           T

In [39]:
mean_xr.isel(time=29, lat = 5, lon = 4).Tair

<xarray.DataArray 'Tair' ()>
array(29.031896)
Coordinates:
    time     float64 2.543e+06
    lon      float32 101.75
    lat      float32 16.25
Attributes:
    long_name:   Near surface air temperature at 2 m at time stamp
    units:       C
    actual_max:  38.56704101562502
    actual_min:  12.980737304687523
    title:       Tair

In [40]:
min_xr.isel(time=29, lat = 5, lon = 4).Tair

<xarray.DataArray 'Tair' ()>
array(23.207605)
Coordinates:
    time     float64 2.543e+06
    lon      float32 101.75
    lat      float32 16.25
Attributes:
    long_name:   Near surface air temperature at 2 m at time stamp
    units:       C
    actual_max:  38.56704101562502
    actual_min:  12.980737304687523
    title:       Tair

In [41]:
max_xr.isel(time=29, lat = 5, lon = 4).Tair

<xarray.DataArray 'Tair' ()>
array(33.369257)
Coordinates:
    time     float64 2.543e+06
    lon      float32 101.75
    lat      float32 16.25
Attributes:
    long_name:   Near surface air temperature at 2 m at time stamp
    units:       C
    actual_max:  38.56704101562502
    actual_min:  12.980737304687523
    title:       Tair